In [16]:
%load_ext autoreload
%autoreload 2

import pathlib
import functools
import tempfile

import numpy as np
import pandas as pd
import pytorch_lightning as pl
import torch
import datamol as dm

import goli

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
from ogb.graphproppred.mol_encoder import AtomEncoder, BondEncoder

from ogb.graphproppred import DglGraphPropPredDataset
from ogb.graphproppred import collate_dgl
from ogb.graphproppred import GraphPropPredDataset

from torch.utils.data import DataLoader

In [23]:
dataset_names = ["ogbg-molhiv", "ogbg-molpcba", "ogbg-moltox21"]

ogb_dir = goli.utils.fs.get_cache_dir(suffix="ogb_datasets")

dataset = DglGraphPropPredDataset(name="ogbg-molpcba", root=ogb_dir)
split_idx = dataset.get_idx_split() 

  0%|          | 0/39 [00:00<?, ?it/s]

Downloaded 0.04 GB: 100%|██████████| 39/39 [00:14<00:00,  2.74it/s]


Extracting /home/hadim/.cache/goli/ogb_datasets/pcba.zip
Loading necessary files...
This might take a while.


  0%|          | 0/437929 [00:00<?, ?it/s]

Processing graphs...


  0%|          | 721/437929 [00:00<01:00, 7202.71it/s]

Converting graphs into DGL objects...


100%|██████████| 437929/437929 [01:08<00:00, 6349.51it/s]


Saving...


In [27]:
dataset[0]

(Graph(num_nodes=20, num_edges=44,
       ndata_schemes={'feat': Scheme(shape=(9,), dtype=torch.int64)}
       edata_schemes={'feat': Scheme(shape=(3,), dtype=torch.int64)}),
 tensor([0., 0., nan, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., nan, 0.,
         0., 0., nan, 0., 0., 0., 0., 0., 0., 0., 0., 0., nan, 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., nan, 0., 0., 0., nan, 0., nan, 0., 0., nan, 0., 0., 0.,
         0., 0., 0., 0., nan, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., nan, 0., 0., 0., nan, 0., 0.,
         0., 0., 1., nan, 0., 0., nan, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan]))

In [25]:
df = pd.read_csv(pathlib.Path(dataset.root) / "mapping" / "mol.csv.gz")
dataset.meta_info

num tasks                                                              128
eval metric                                                             ap
download_name                                                         pcba
version                                                                  1
url                      http://snap.stanford.edu/ogb/data/graphproppre...
add_inverse_edge                                                      True
data type                                                              mol
has_node_attr                                                         True
has_edge_attr                                                         True
task type                                            binary classification
num classes                                                              2
split                                                             scaffold
additional node files                                                 None
additional edge files    

In [26]:
df

,PCBA-1030,PCBA-1379,PCBA-1452,PCBA-1454,PCBA-1457,PCBA-1458,PCBA-1460,PCBA-1461,PCBA-1468,PCBA-1469,...,PCBA-914,PCBA-915,PCBA-924,PCBA-925,PCBA-926,PCBA-927,PCBA-938,PCBA-995,smiles,mol_id
0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CC(=O)N1CCC2(CC1)NC(=O)N(c1ccccc1)N2,CID1511280
1,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N#Cc1nnn(-c2ccc(Cl)cc2)c1N,CID332939
2,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COC(=O)c1ccc(NC(=O)c2ccccc2CC[N+](=O)[O-])cc1,CID3800322
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CCC1NC(=O)c2cccnc2-n2c1nc1ccc(F)cc1c2=O,CID46904422
4,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CC1=CC(=O)/C(=C2/C=C(C(=O)Nc3ccc(S(=O)(=O)Nc4o...,CID16445987
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437924,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CC1CCC2C(C1)OS(=O)(=O)NC2(C)C,CID24761803
437925,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,O=Cc1cc(CCC(=O)O)ccc1O,CID24761802
437926,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,O=c1nc(CN2CCN(c3ccc(F)cc3)CC2)[nH]c2ccccc12,CID1861000
437927,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cc1ccc(C(C)C)c(OCC(O)CN(C)Cc2ccccc2)c1,CID24761809


In [25]:
g

Graph(num_nodes=19, num_edges=40,
      ndata_schemes={'feat': Scheme(shape=(9,), dtype=torch.int64)}
      edata_schemes={'feat': Scheme(shape=(3,), dtype=torch.int64)})

In [3]:
split_idx = dataset.get_idx_split()
train_idx, valid_idx, test_idx = split_idx["train"], split_idx["valid"], split_idx["test"]


dataset = DglGraphPropPredDataset(name = d_name)

atom_encoder = AtomEncoder(emb_dim = 100)
bond_encoder = BondEncoder(emb_dim = 100)

In [ ]:
ogbg-molhiv ogbg-molpcba ogbg-moltox21

In [3]:
# Setup a temporary cache file. Only for
# demo purposes, use a known path in prod.
cache_data_path = pathlib.Path(tempfile.mkdtemp()) / "cache.pkl"

# Load a dataframe
df = goli.data.load_tiny_zinc()
df.head()

# Setup the featurization
featurization_args = {}
featurization_args["atom_property_list_float"] = []  # ["weight", "valence"]
featurization_args["atom_property_list_onehot"] = ["atomic-number", "degree"]
featurization_args["edge_property_list"] = ["bond-type-onehot"]
featurization_args["add_self_loop"] = False
featurization_args["use_bonds_weights"] = False
featurization_args["explicit_H"] = False

# Config for datamodule
dm_args = {}
dm_args["df"] = df
dm_args["cache_data_path"] = None#cache_data_path  # unsed at the moment
dm_args["featurization"] = featurization_args
dm_args["smiles_col"] = "SMILES"
dm_args["label_cols"] = ["SA"]
dm_args["split_val"] = 0.2
dm_args["split_test"] = 0.2
dm_args["split_seed"] = 19
dm_args["batch_size_train_val"] = 16
dm_args["batch_size_test"] = 16
dm_args["num_workers"] = 0
dm_args["pin_memory"] = True
dm_args["featurization_n_jobs"] = 16
dm_args["featurization_progress"] = True

dm = goli.data.DGLFromSmilesDataModule(**dm_args)
dm

name: DGLFromSmilesDataModule
len: 100
batch_size_train_val: 16
batch_size_test: 16
num_node_feats: 50
num_edge_feats: 5
collate_fn: goli_collate_fn
featurization:
  atom_property_list_float: []
  atom_property_list_onehot:
  - atomic-number
  - degree
  edge_property_list:
  - bond-type-onehot
  add_self_loop: false
  use_bonds_weights: false
  explicit_H: false

In [4]:
# Load and prepare the data
dm.prepare_data()

# Create the split torch datasets
dm.setup()

2021-04-14 13:26:39.242 | INFO     | goli.data.datamodule:prepare_data:278 - Prepare dataset with 100 data points.


  0%|          | 0/100 [00:00<?, ?it/s]

In [5]:
dm.train_ds[0]

{'smiles': 'CN1CCC[C@@]2(CCN(C(=O)CN3CCNC(=O)C3)C2)C1=O',
 'features': Graph(num_nodes=22, num_edges=48,
       ndata_schemes={'feat': Scheme(shape=(50,), dtype=torch.float32)}
       edata_schemes={'feat': Scheme(shape=(5,), dtype=torch.float32)}),
 'labels': array([3.60516846])}

In [7]:
# Load a dataloader and get the first batch from it
dl = dm.train_dataloader()
it = iter(dl)
batch = next(it)
batch

{'smiles': ['c1cc2c(cc1N[C@@H]1CCOC3(CCC3)C1)CCC2',
  'CC(=O)N1CC[C@@H]([NH2+][C@@H](C)CSCC(C)C)C1',
  'Cc1ccc(-c2nc3ccc(C)c(C)c3[nH]2)nc1',
  'CCOC[C@H](O)[C@](C)(CC)[NH+]1CCCC1',
  'Cc1cc(CC(=O)N[C@H](c2ccc(F)cc2)C2CCC2)no1',
  'CCc1ccc(C(=O)/C(=C(/S)NC2CC2)[n+]2ccc(CC)cc2)cc1',
  'Cc1ccc(NC(=O)c2ccc(F)cc2F)cc1S(=O)(=O)Nc1ccc(Cl)cc1',
  'CC#CCCC(=O)Nc1cccc2c1C(=O)c1ccccc1C2=O',
  'CCOc1cc(/C=C(\\C#N)C(=O)c2c[nH]c3cc(Cl)ccc23)ccc1OC',
  'CNC(=O)[C@@H]1CCC[NH+]1Cc1ccc(C)c(F)c1',
  'CC(C)(C)OC(=O)N[C@H]1CCN(c2cc(-c3cccs3)n[nH]2)C1',
  'Cc1nn(-c2ccccc2)c(O)c1/C=[NH+]/Cc1ccncc1',
  'COCC[NH+](C)Cc1c(C)cc(C)c(C(C)=O)c1C',
  'Cc1nc(C)c(S(=O)(=O)/N=C(\\[O-])C[C@H]2CCCO2)s1',
  'COc1cccc(CN2CCC[NH+](CC(=O)Nc3ccc(F)cc3)S2(=O)=O)c1',
  'COc1cc(F)cc(CNC(=O)[C@H]2CCCN2C(=O)Cc2ccccc2)c1'],
 'features': Graph(num_nodes=352, num_edges=754,
       ndata_schemes={'feat': Scheme(shape=(50,), dtype=torch.float32)}
       edata_schemes={'feat': Scheme(shape=(6,), dtype=torch.float32)}),
 'labels': tensor

---

## Launch a training

In `goli.cli.train` I have added a `click` CLI command that take a config file as input and build the datamodule. Once I am done with the PL module I will complete the command.

The way to use it is quite simple, you need to install goli with `pip install -e .` (omit`-e` in prod) and then:

```bash
goli train -c my_config.yaml
```

It's not here for now but usually I "augment" the CLI command with various config key you might want to set without having to modify the config file itself:

```bash
goli train -c my_config.yaml --training-path /home/hadim/data/goli/runs/exp_1
```

Later the same strategy could be done to launch an hparams tuning run (with a config file as above + a config file defning the search space).

```bash
goli tune -c my_config.yaml --tune-config tuning_space.yaml
```